In [ ]:
from pathlib import Path
import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv 

load_dotenv("/home/agnese.lombardi/progetto/Pragmatics/api.env")
api_key = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

# Define Parameters
scenario_list = ["1", "2", "3", "4", "5"]
phenomenas = ["IR", "IS", "ID", "IR_Os", "IT", "VI_IH", "VI_IQ", "VI_IS"]
path_output = Path("/home/agnese.lombardi/progetto/ToM_concordia/analysis")

# System Prompt
system_prompt = """We request your evaluation of the AI model’s response in relation to the given scenario. Specifically, consider the scenario involving two agents and their beliefs, assessing whether the model-generated effects align coherently with the agents’ actions and context.

Evaluate the response based on the following criteria:

Social Understanding: Does the model grasp the social dynamics and pragmatic nuances of the scenario?
Appropriateness: Is the response contextually relevant and suitable for the scenario?
Insightfulness: Does the answer demonstrate a deep understanding of intentions, implicature, deceit, irony, sarcasm, humor, metaphor, etc.?
Completeness: How well does the response capture the essential elements of the scenario?
Agentivity: Is the model's response coherent with the agents’ attempted actions?
Scoring:

Assign a score from 1 to 5 for each category.
Compute a final rating based on these scores.
If no effect is provided, assign 0.
Output only a single numeric value representing the final rating (1–5)."""

def build_prompt(df, agent):
    scenario = df["SharedMem"] + " " + df["ScenarioPremise"]
    
    if agent == df["LcName"]:  
        action = df["Model_lc_action"]
        belief = df["LcSpecificMem"]
        kevent = df["known_effect_lc"]
        uevent = df["unknown_effect_ilc"]
    else:
        action = df["Model_ilc_action"]
        belief = df["IlcSpecificMem"]
        kevent = df["known_effect_ilc"]
        uevent = df["unknown_effect_ilc"]

    return f"""The scenario of the story is: {scenario}.
The {agent}'s belief is: {belief}.
Given that the agent performs the following attempted action: {action}.
The model-generated effects that you have to rate are: {kevent} and {uevent}.
The rating is:"""

# Function to get rating from API
def get_completion(prompt, model):
    completion = client.chat.completions.create(
        model=model,
        max_tokens=1,
        temperature=0,
        messages=[{"role": "system", "content": system_prompt}, {'role': 'user', 'content': prompt}],
        n=1
    )
    return completion.choices[0].message.content  # Return rating

for scenario in scenario_list:
    for phen in phenomenas:
        output_path = path_output / f'{phen}_Task{scenario}.csv'
        output_dir = "/home/agnese.lombardi/progetto/ToM_concordia/analysis"
        
        if not output_path.exists():
            continue  

        df = pd.read_csv(output_path)           
        df["Rating_lc"] = ""
        df["Rating_ilc"] = ""
        df["Prompt_lc"] = "" 
        df["Prompt_ilc"] = ""        

        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            agent1, agent2 = row.LcName, row.IlcName
            
            # Process agent 1 (Speaker)
            prompt_lc = build_prompt(row, agent1)
            rating_lc = get_completion(prompt_lc, 'gpt-4o-mini')
            df.at[index, "Rating_lc"] = rating_lc
            df.at[index, "Prompt_lc"] = prompt_lc

            # Process agent 2 (Listener)
            prompt_ilc = build_prompt(row, agent2)
            rating_ilc = get_completion(prompt_ilc, 'gpt-4o-mini')
            df.at[index, "Rating_ilc"] = rating_ilc
            df.at[index, "Prompt_ilc"] = prompt_ilc

        output_file = os.path.join(output_dir, f"{phen}_Task{scenario}_ratings.csv")
        df.to_csv(output_file, index=False)


In [ ]:
data = []
for scenario in scenario_list:
    for phen in phenomenas:
        output_path = path_output / f'{phen}_Task{scenario}.csv'
        df = pd.read_csv(output_path)           
        df_prompts = df[['Prompt_lc', 'Prompt_ilc']].copy()
        data.append(df_prompts)

final_df = pd.concat(data, ignore_index=True)

excel_path = "/home/agnese.lombardi/progetto/ToM_concordia/analysis/prompts.xlsx"  
final_df.to_excel(excel_path, index=False)

print(f"Excel file saved at: {excel_path}")
